# Cleaning Tweets - NLP

### Importing Libraries

In [33]:
import numpy as np
import random
import _pickle as pickle

### Importing DataFrames

In [34]:
# Loading in the first DF
with open("city1_df.pkl",'rb') as fp:
    city_1 = pickle.load(fp)

# Loading in the second DF
with open("city2_df.pkl",'rb') as fp:
    city_2 = pickle.load(fp)

### Checking the DataFrames

In [35]:
city_1

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,link,...,search,source,timezone,tweet,user_id,user_id_str,user_rt,user_rt_id,username,City
0,[],1179398288778366977,1570028726000,2019-10-02 11:05:26,3,,"[#pelosi, #schiff, #trumpimpeachmentinquiry]",14,1179412079830327296,https://twitter.com/cynthia_kirk03/status/1179...,...,Trump,,EDT,SCANDAL RANGERS TO THE RESCUE! I'm proud to be...,836002063137951744,836002063137951744,,,cynthia_kirk03,Seattle
1,[],1179138038464540674,1570028726000,2019-10-02 11:05:26,3,,[],14,1179412079066992644,https://twitter.com/k8briz/status/117941207906...,...,Trump,,EDT,Well-what is it?,755855317246672896,755855317246672896,,,k8briz,Seattle
2,[],1179411261337067520,1570028697000,2019-10-02 11:04:57,3,,[],06,1179411954416439296,https://twitter.com/mfow020/status/11794119544...,...,Trump,,EDT,Enemy of the people!!!,987789810570969088,987789810570969088,,,mfow020,Seattle
3,[],1179411261337067520,1570028692000,2019-10-02 11:04:52,3,,[],05,1179411936582209536,https://twitter.com/dougalpollux/status/117941...,...,Trump,,EDT,Top of the morning 😃 Mr President,226099935,226099935,,,dougalpollux,Seattle
4,[],1179196668023054336,1570028671000,2019-10-02 11:04:31,2,,[],23,1179411847642062851,https://twitter.com/Craterdome/status/11794118...,...,Trump,,EDT,Is there any line Trump can cross that would m...,15481999,15481999,,,Craterdome,Seattle
5,[],1179407402933870592,1570028665000,2019-10-02 11:04:25,2,,[],21,1179411820974628871,https://twitter.com/TamaraZRoberson/status/117...,...,Trump,,EDT,So Trump and his supporters threatening to kil...,1028106497237049344,1028106497237049344,,,TamaraZRoberson,Seattle
6,[],1179411261337067520,1570028661000,2019-10-02 11:04:21,2,,[],20,1179411803417264136,https://twitter.com/mfow020/status/11794118034...,...,Trump,,EDT,America needs someone who is that tough!!! Res...,987789810570969088,987789810570969088,,,mfow020,Seattle
7,[],1179411764573872129,1570028651000,2019-10-02 11:04:11,2,,[],17,1179411764573872129,https://twitter.com/ColinDavies1967/status/117...,...,Trump,,EDT,When Trump and his allies finally hit rock bot...,824303270407512064,824303270407512064,,,ColinDavies1967,Seattle
8,[],1179411764074762240,1570028651000,2019-10-02 11:04:11,2,,[#dontheconjr],17,1179411764074762240,https://twitter.com/TheMindOfColi/status/11794...,...,Trump,,EDT,"seriously, i “thought” he was supposed to be t...",975209966,975209966,,,TheMindOfColi,Seattle
9,[],1179411261337067520,1570028644000,2019-10-02 11:04:04,2,,[],15,1179411736170029056,https://twitter.com/Fragola_Girl/status/117941...,...,Trump,,EDT,Nope. Moot. pic.twitter.com/hwbHpQ6OQ0,1284836701,1284836701,,,Fragola_Girl,Seattle


In [36]:
city_2

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,link,...,search,source,timezone,tweet,user_id,user_id_str,user_rt,user_rt_id,username,City
0,[],1179411261337067520,1570029021000,2019-10-02 11:10:21,7,,[],00,1179413316420153347,https://twitter.com/SuZiRok/status/11794133164...,...,Trump,,EDT,You can always build AI one's.,166775935,166775935,,,SuZiRok,Jacksonville
1,[],1179378450936807424,1570028948000,2019-10-02 11:09:08,6,,[],04,1179413009929838599,https://twitter.com/EmmyDee4America/status/117...,...,Trump,,EDT,"Do several rallies there, sir. I know there ar...",880625263096336384,880625263096336384,,,EmmyDee4America,Jacksonville
2,[],1179404929749934081,1570028823000,2019-10-02 11:07:03,4,,[],17,1179412485759209472,https://twitter.com/TreyTackling/status/117941...,...,Trump,,EDT,"Looking past nomination, this plays right into...",66007392,66007392,,,TreyTackling,Jacksonville
3,[],1179399836883394562,1570028772000,2019-10-02 11:06:12,4,,[],03,1179412271304454153,https://twitter.com/EmmyDee4America/status/117...,...,Trump,,EDT,Agreed. He’s got to go..... to prison.,880625263096336384,880625263096336384,,,EmmyDee4America,Jacksonville
4,[],1179411261337067520,1570028740000,2019-10-02 11:05:40,3,,"[#treasonoustrump, #guilty]",18,1179412134968614913,https://twitter.com/JustJeffOmaha/status/11794...,...,Trump,,EDT,Had to re-do this after initially spelling Moa...,29440131,29440131,,,JustJeffOmaha,Jacksonville
5,[],1179411261337067520,1570028722000,2019-10-02 11:05:22,3,,[],13,1179412061320859649,https://twitter.com/SuZiRok/status/11794120613...,...,Trump,,EDT,"If you're going to build a wall, might as well...",166775935,166775935,,,SuZiRok,Jacksonville
6,[],1179411261337067520,1570028682000,2019-10-02 11:04:42,3,,[],02,1179411892709793792,https://twitter.com/EmmyDee4America/status/117...,...,Trump,,EDT,I’m not opposed to it. Tired of human & drug t...,880625263096336384,880625263096336384,,,EmmyDee4America,Jacksonville
7,[],1179411261337067520,1570028681000,2019-10-02 11:04:41,3,,[],02,1179411889245343745,https://twitter.com/JendySharp/status/11794118...,...,Trump,,EDT,I preferred Moot,32252758,32252758,,,JendySharp,Jacksonville
8,[],1179409196191801347,1570028622000,2019-10-02 11:03:42,2,,[],09,1179411639944265728,https://twitter.com/DonPolka1/status/117941163...,...,Trump,,EDT,"Not even a ""shell"" of what he once was at the ...",1052732864289095680,1052732864289095680,,,DonPolka1,Jacksonville
9,[],1179411536575713280,1570028597000,2019-10-02 11:03:17,2,,[],02,1179411536575713280,https://twitter.com/KimSchwan/status/117941153...,...,Trump,,EDT,Shout out to @KristenBell for giving us this p...,32744001,32744001,,,KimSchwan,Jacksonville


## Cleaning and NLP

In [37]:
def city_balance(city_1, city_2):
    """
    Balances the number of unique tweets from each city
    """
    # Removing tweets from the same user
    city_1 = city_1.drop_duplicates('user_id')
    city_2 = city_2.drop_duplicates('user_id')

    # Checking for class balance
    dif = abs(len(city_1) - len(city_2))

    # Randomly dropping rows from the DF with a greater number of rows
    if len(city_1) > len(city_2):
        city_1 = city_1.drop(random.sample(list(city_1.index), dif))
    elif len(city_2) > len(city_1):
        city_2 = city_2.drop(random.sample(list(city_2.index), dif))
    else:
        print("DFs are balanced")
    
    print(f"DF Lengths:\nCity 1 = {len(city_1)}\nCity 2 = {len(city_2)}")

    return city_1, city_2

In [38]:
city_1, city_2 = city_balance(city_1, city_2)

DF Lengths:
City 1 = 952
City 2 = 952


In [39]:
# Combining both dataframes
main_df = city_1.append(city_2)
main_df

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,link,...,search,source,timezone,tweet,user_id,user_id_str,user_rt,user_rt_id,username,City
1,[],1179138038464540674,1570028726000,2019-10-02 11:05:26,3,,[],14,1179412079066992644,https://twitter.com/k8briz/status/117941207906...,...,Trump,,EDT,Well-what is it?,755855317246672896,755855317246672896,,,k8briz,Seattle
2,[],1179411261337067520,1570028697000,2019-10-02 11:04:57,3,,[],06,1179411954416439296,https://twitter.com/mfow020/status/11794119544...,...,Trump,,EDT,Enemy of the people!!!,987789810570969088,987789810570969088,,,mfow020,Seattle
3,[],1179411261337067520,1570028692000,2019-10-02 11:04:52,3,,[],05,1179411936582209536,https://twitter.com/dougalpollux/status/117941...,...,Trump,,EDT,Top of the morning 😃 Mr President,226099935,226099935,,,dougalpollux,Seattle
7,[],1179411764573872129,1570028651000,2019-10-02 11:04:11,2,,[],17,1179411764573872129,https://twitter.com/ColinDavies1967/status/117...,...,Trump,,EDT,When Trump and his allies finally hit rock bot...,824303270407512064,824303270407512064,,,ColinDavies1967,Seattle
11,[],1179391179126906880,1570028639000,2019-10-02 11:03:59,2,,[],14,1179411715131367424,https://twitter.com/SecTechWorker/status/11794...,...,Trump,,EDT,A direct call to restrict free speech would be...,1128838635233484800,1128838635233484800,,,SecTechWorker,Seattle
13,[],1179159038937853952,1570028634000,2019-10-02 11:03:54,2,,[],13,1179411693342003201,https://twitter.com/questeverything/status/117...,...,Trump,,EDT,"As @maddow pointed out yesterday, predatory pl...",309353994,309353994,,,questeverything,Seattle
15,[],1179411261337067520,1570028600000,2019-10-02 11:03:20,2,,[],03,1179411547904516097,https://twitter.com/twittdwitt/status/11794115...,...,Trump,,EDT,Too late. Moot’s out there.,37804799,37804799,,,twittdwitt,Seattle
18,[],1179195929154461696,1570028592000,2019-10-02 11:03:12,2,,[],01,1179411516971507713,https://twitter.com/HelloooKittyH3/status/1179...,...,Trump,,EDT,"I’ve never been a fan of him as a person, but ...",922124211283304448,922124211283304448,,,HelloooKittyH3,Seattle
19,[],1178986524630802432,1570028588000,2019-10-02 11:03:08,2,,[],00,1179411501062467589,https://twitter.com/WillSeattle/status/1179411...,...,Trump,,EDT,More communism from Trump,16645396,16645396,,,WillSeattle,Seattle
25,[],1175527658551173120,1570028532000,2019-10-02 11:02:12,1,,[],08,1179411264390467591,https://twitter.com/TerriSeattle/status/117941...,...,Trump,,EDT,"Dude, give it up.",943662167936655360,943662167936655360,,,TerriSeattle,Seattle


## Word Embeddings